In [6]:
import gym
import numpy as np

MAX_NUM_EPISODES = 50000
STEPS_PER_EPISODE = 200 #  This is specific to MountainCar. May change with env
EPSILON_MIN = 0.005
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
EPSILON_DECAY = 500 * EPSILON_MIN / max_num_steps
ALPHA = 0.05  # Learning rate
GAMMA = 0.98  # Discount factor
NUM_DISCRETE_BINS = 30  # Number of bins to Discretize each observation dim



In [5]:
# import torch
# torch.cuda.is_available()

In [51]:
class QAgent(object):
    def __init__(self):
        self.obs_shape = env.observation_space.shape
        self.obs_high = env.observation_space.high
        self.obs_low = env.observation_space.low
        self.obs_bins = NUM_DISCRETE_BINS  # Number of bins to Discretize each observation dim
        self.bin_width = (self.obs_high - self.obs_low) / self.obs_bins
        self.action_shape = env.action_space.n
        # Create a multi-dimensional array (aka. Table) to represent the
        # Q-values
        self.Q = np.zeros((self.obs_bins + 1, self.obs_bins + 1,
                          self.action_shape))
        self.alpha = ALPHA  # Learning rate
        self.gamma = GAMMA  # Discount factor
        self.epsilon = 1.0
        
    def discretize(self, obs):
        return tuple(((obs - self.obs_low) / self.bin_width).astype(int))
    
    def get_action(self, obs):
        discretized_obs = self.discretize(obs)
        # Epsilon-Greedy action selection
        if self.epsilon > EPSILON_MIN:
            self.epsilon -= EPSILON_DECAY
        if np.random.random() > self.epsilon:
            return np.argmax(self.Q[discretized_obs])
        else:  # Choose a random action
            return np.random.choice([a for a in range(self.action_shape)])
        
    def learn(self, obs, action, reward, next_obs):
        discretized_obs = self.discretize(obs)
        discretized_next_obs = self.discretize(next_obs)
        td_target = reward + self.gamma * np.max(self.Q[discretized_next_obs])
        td_error = td_target - self.Q[discretized_obs][action]
        self.Q[discretized_obs][action] += self.alpha * td_error
    
    

In [11]:
env = gym.make('MountainCar-v0')
np.zeros((31, 31, env.action_space.n)).shape

(31, 31, 3)

In [12]:
env.action_space.n

3

In [16]:
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [44]:
obs = env.reset()

In [49]:
obs

array([-0.49453864,  0.        ], dtype=float32)

In [26]:
q = QAgent()

In [50]:
q.discretize(obs)

(11, 15)

In [53]:
def train(agent, env):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        done = False
        obs = env.reset()
        total_reward = 0.0
        while not done:
            action = agent.get_action(obs)
            next_obs, reward, done, info = env.step(action)
            agent.learn(obs, action, reward, next_obs)
            obs = next_obs
            total_reward += reward
        if total_reward > best_reward:
            best_reward = total_reward
        print("Episode#:{} reward:{} best_reward:{} eps:{}".format(episode,
                                     total_reward, best_reward, agent.epsilon))
    # Return the trained policy
    return np.argmax(agent.Q, axis=2)

In [54]:
def test(agent, env, policy):
    done = False
    obs = env.reset()
    total_reward = 0.0
    while not done:
        action = policy[agent.discretize(obs)]
        next_obs, reward, done, info = env.step(action)
        obs = next_obs
        total_reward += reward
    return total_reward

In [ ]:
env = gym.make('MountainCar-v0')
agent = QAgent()
learned_policy = train(agent, env)
    # Use the Gym Monitor wrapper to evalaute the agent and record video
gym_monitor_path = "./gym_monitor_output"
env = gym.wrappers.Monitor(env, gym_monitor_path, force=True)
for _ in range(1000):
    test(agent, env, learned_policy)
env.close()

Episode#:0 reward:-200.0 best_reward:-200.0 eps:0.999949999999993
Episode#:1 reward:-200.0 best_reward:-200.0 eps:0.999899999999986
Episode#:2 reward:-200.0 best_reward:-200.0 eps:0.999849999999979
Episode#:3 reward:-200.0 best_reward:-200.0 eps:0.999799999999972
Episode#:4 reward:-200.0 best_reward:-200.0 eps:0.999749999999965
Episode#:5 reward:-200.0 best_reward:-200.0 eps:0.9996999999999581
Episode#:6 reward:-200.0 best_reward:-200.0 eps:0.9996499999999511
Episode#:7 reward:-200.0 best_reward:-200.0 eps:0.9995999999999441
Episode#:8 reward:-200.0 best_reward:-200.0 eps:0.9995499999999371
Episode#:9 reward:-200.0 best_reward:-200.0 eps:0.9994999999999301
Episode#:10 reward:-200.0 best_reward:-200.0 eps:0.9994499999999231
Episode#:11 reward:-200.0 best_reward:-200.0 eps:0.9993999999999161
Episode#:12 reward:-200.0 best_reward:-200.0 eps:0.9993499999999091
Episode#:13 reward:-200.0 best_reward:-200.0 eps:0.9992999999999022
Episode#:14 reward:-200.0 best_reward:-200.0 eps:0.999249999999